In [1]:
from guidance import models, gen, select
import guidance
lm = models.Transformers('TheBloke/Nous-Hermes-2-SOLAR-10.7B-GPTQ', device_map="cuda", echo=True)

import chromadb
from sentence_transformers import SentenceTransformer
#sentences = ["This is an example sentence", "Each sentence is converted"]

sentence_transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

import time
from datetime import date

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\User\miniconda3\envs\Lucid\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:

#embeddings = sentence_transformer.encode(sentences)#
short_term_memory = chromadb.Client()
#client = chromadb.PersistentClient(path="./Chroma")
working_memory = []
"""
This is what a Info Block should look like
{
	'object_type' : 'people',
	'object_name' : 'Miko',
	'content' : 'Miko like Nintendo games.',
	'timestamp' : '2021-10-15 17:37:00',
	'vector' array([[-4.39221077e-02, -1.25277145e-02,  2.93133650e-02,]], dtype=float32): ,
}    

with guidance
f'''\
{
	'object_type' : '{select(['people','events'])'},
	'object_name' : '{gen(stop='\'')},
	'content' : '{gen(stop='\'')},
	'timestamp' : {date.fromtimestamp(time.time())},
	'vector' : {sentence_transformer.encode([])},
}'''
"""
#def process_new_info_block(lm_text_result):

@guidance(stateless=True)
def guidance_make_new_info_block(lm, passage):
		first_curly = "{"
		second_curly = "}"
		lm += """\
This is what a Info Block should look like from an example:
[Example]
Lucid: What are you doing now? Stop ignoring me!
Miko: I'm playing Nintendo games.
Lucid: And that's a higher priority than me?
Miko: ...
Lucid: Your silence speaks volumes.

```json
{
	"object_type" : 'people',
	"object_name" : 'Miko',
	"content" : 'Miko likes Nintendo games.',
}
```
[End of Example]
"""
		lm += f"""\
{passage}
```json
{first_curly}
	"object_type" : '{select(['people','events'], name="object_type")},
	"object_name" : "{gen(stop='"',name = "object_name")},
	"content" : "{gen(stop='"',name="content")},"""  
		return lm

def make_new_info_block(lm, passage):
	first_curly = "{"
	second_curly = "}"
	lm += guidance_make_new_info_block(passage)
	print(lm['object_type'],lm['object_name'],lm['content'])
	timestamp = date.fromtimestamp(time.time())
	json_file ={
	"object_type" : lm['object_type'],
	"object_name" : lm['object_name'],
	"content" : lm['content'],
	"timestamp" : timestamp,
	"vector" : sentence_transformer.encode([lm['content']]),
	}
	return json_file

In [4]:
passage = """\
Lucid: Hey Miko what's your favorite food?
Miko: Umm I don't know maybe rice?
Lucid you are really boring you know?"""
make_new_info_block(lm, passage)

people Miko Miko like rice.


{'object_type': 'people',
 'object_name': 'Miko',
 'content': 'Miko like rice.',
 'timestamp': datetime.date(2024, 2, 18),
 'vector': array([[-4.08525020e-02, -2.68578772e-02,  8.09861161e-03,
          4.50772159e-02, -1.44550763e-02, -5.42962663e-02,
          1.14020392e-01,  7.10954238e-03, -3.50764790e-03,
         -7.66906561e-03,  1.08682312e-01, -4.24704254e-02,
         -5.17556630e-02,  4.34186459e-02, -1.17587242e-02,
          4.46850732e-02,  8.84275511e-02,  3.11143026e-02,
         -5.65551147e-02, -3.25349160e-02, -2.91228760e-02,
         -4.35660109e-02,  3.27593051e-02,  2.98951613e-03,
          3.74890072e-03, -3.21883038e-02,  4.02739123e-02,
          6.45180494e-02,  3.43699660e-03, -2.01022625e-02,
         -2.39502341e-02,  8.39118883e-02,  5.52131608e-03,
         -2.28218306e-02, -2.15481967e-02,  9.85663235e-02,
          6.56419769e-02, -5.48602380e-02,  5.24938339e-03,
         -4.46575731e-02, -4.19697948e-02, -7.26909712e-02,
          6.56801686e-02,  